In [1]:
from datetime import datetime
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
from elasticsearch.helpers import bulk
es = Elasticsearch(['atlas-kibana.mwt2.org:9200'])

In [2]:
ind = "network_weather_2-2015.12.12"

my_query = \
{
  "size": 0,
  "aggs": {      
        "type_counts": {
            "terms": { "field": "_type" },
            "aggs": {      
                "src_counts": {
                     "terms": { "field": "src" , "size": 10000},
                     "aggs": {      
                        "dest_counts": {
                        "terms": { "field": "dest" , "size": 10000}
                        }
                     }
                  }
              }
        }
  }
}

In [3]:
response = es.search(index=ind, body=my_query, request_timeout=600)

In [4]:
#print(response['aggregations']['type_counts']['buckets'])


In [5]:

types = response['aggregations']['type_counts']['buckets']

for t in types:
    print(t['key'])
    for s in t['src_counts']['buckets']:
        for d in s['dest_counts']['buckets']:
            src=s['key']
            dest=d['key']
            print ('s: ',src,'d: ', dest, '\t docs:', d['doc_count'])
            myscroll={
            "size":100000,
#             "fields": ["timestamp", "delay_mean","throughput","packet_loss","src","dest"],
            "fields": ["timestamp"],
            "query":{
                "filtered": {
                    "query": {
                         "match_all": {} 
                        },
                        "filter" : { 
                            "bool" : {
                                "must" : [
                            {"term": {"_type": {"value": t['key'] } } },
                            {"term": {"src":   {"value": src } } },
                            {"term": {"dest":  {"value": dest } } }
                                ]
                            }

                        }
                }
            }
            ,"sort": [{ "timestamp": { "order": "asc" }}]
            }
            
            toDelete={}
#             oldres=0
            oldtime=""
            counter=0
            scroll = scan(es, query=myscroll, index=ind, scroll='120s')
            for res in scroll:
#                 if counter==0: print(res)
                if res['fields']['timestamp'][0]==oldtime:# and res['fields']['src'][0]==src and res['fields']['dest'][0]==dest and res['fields']['packet_loss'][0]==oldres:
                    toDelete[res['_id']]=res['_index']
#                     print(res['fields'])
#                 oldres=res['fields']['packet_loss'][0]
                oldtime=res['fields']['timestamp'][0]
                counter+=1
#                if counter>200:
#                    break
#                 if counter%100==0: 
                
            print ("records:", counter, '\tto delete:',len(toDelete))
            bulk_body = []
            for x in toDelete:
                bulk_body.append('{"delete": {"_id": "'+x+'", "_index":"'+toDelete[x]+'", "_type":"'+t['key']+'"}}')
#             print (bulk_body)
            if len(toDelete):
                es.bulk('\n'.join(bulk_body))

packet_loss_rate
s:  85.122.31.74 d:  146.83.90.7 	 docs: 4825
records: 4825 	to delete: 4799
s:  85.122.31.74 d:  128.40.4.25 	 docs: 4356
records: 4356 	to delete: 4317
s:  85.122.31.74 d:  81.180.86.38 	 docs: 4085
records: 4085 	to delete: 4008
s:  85.122.31.74 d:  131.154.254.12 	 docs: 2199
records: 2199 	to delete: 1988
s:  85.122.31.74 d:  117.103.105.191 	 docs: 2121
records: 2121 	to delete: 2067
s:  85.122.31.74 d:  147.213.204.112 	 docs: 1854
records: 1854 	to delete: 1762
s:  85.122.31.74 d:  132.206.245.252 	 docs: 1840
records: 1840 	to delete: 1826
s:  85.122.31.74 d:  138.253.60.82 	 docs: 1838
records: 1838 	to delete: 1747
s:  85.122.31.74 d:  134.158.103.10 	 docs: 1799
records: 1799 	to delete: 1574
s:  85.122.31.74 d:  141.108.35.18 	 docs: 1773
records: 1773 	to delete: 1669
s:  85.122.31.74 d:  139.184.80.18 	 docs: 1523
records: 1523 	to delete: 1411
s:  85.122.31.74 d:  163.1.5.210 	 docs: 1523
records: 1523 	to delete: 1415
s:  85.122.31.74 d:  169.228.130.4

ConnectionTimeout: ConnectionTimeout caused by - ReadTimeoutError(HTTPConnectionPool(host='atlas-kibana.mwt2.org', port=9200): Read timed out. (read timeout=10))